# Gapfillling

Model gap filling is the task of figuring out which reactions have to be added to a model to make it feasible. Several such algorithms have been reported e.g. [Kumar et al. 2009](http://dx.doi.org/10.1371/journal.pcbi.1000308) and [Reed et al. 2006](http://www.pnas.org/content/103/46/17480.short). Cobrapy has a gap filling implementation that is very similar to that of Reed et al. where we use a mixed-integer linear program to figure out the smallest number of reactions that need to be added for a user-defined collection of reactions, i.e. a universal model. Briefly, the problem that we try to solve is

Minimize: $$\sum_i c_i * z_i$$

subject to

$$Sv = 0$$
$$v^\star \geq t$$
$$l_i\leq v_i \leq u_i$$
$$v_i = 0 \textrm{ if } z_i = 0$$

Where *l*, *u* are lower and upper bounds for reaction *i* and *z* is an indicator variable that is zero if the reaction is not used and otherwise 1, *c* is a user-defined cost associated with using the *i*th reaction, $v^\star$ is the flux of the objective and *t* a lower bound for that objective. To demonstrate, let's take a model and remove some essential reactions from it.

In [3]:
from cobra.io import load_model
from cobra.flux_analysis import gapfill
model = load_model("iYS1720")

In this model D-Fructose-6-phosphate is an essential metabolite. We will remove all the reactions using it, and at them to a separate model.

In [7]:
import cobra
universal = cobra.Model("universal_reactions")
for i in [i.id for i in model.metabolites.f6p_c.reactions]:
    reaction = model.reactions.get_by_id(i)
    universal.add_reactions([reaction.copy()])
    model.remove_reactions([reaction])

Now, because of these gaps, the model won't grow.

In [8]:
model.optimize().objective_value

0.0

We will use can use the model's original objective, growth, to figure out which of the removed reactions are required for the model be feasible again. This is very similar to making the 'no-growth but growth (NGG)' predictions of [Kumar et al. 2009](http://dx.doi.org/10.1371/journal.pcbi.1000308).

In [10]:
solution = gapfill(model, universal, demand_reactions=False)
for reaction in solution[0]:
    print(reaction.id)

Read LP format model from file /var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/tmpshctvlc5.lp
Reading time = 0.02 seconds
: 2436 rows, 6684 columns, 24996 nonzeros
Read LP format model from file /var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/tmpw7ur2bdf.lp
Reading time = 0.00 seconds
: 28 rows, 30 columns, 118 nonzeros
GF6PTA
TALA


We can obtain multiple possible reaction sets by having the algorithm go through multiple iterations.

In [11]:
result = gapfill(model, universal, demand_reactions=False, iterations=4)
for i, entries in enumerate(result):
    print("---- Run %d ----" % (i + 1))
    for e in entries:
        print(e.id)

Read LP format model from file /var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/tmp1wn5o8dy.lp
Reading time = 0.02 seconds
: 2436 rows, 6684 columns, 24996 nonzeros
Read LP format model from file /var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/tmpa1kn_gtf.lp
Reading time = 0.00 seconds
: 28 rows, 30 columns, 118 nonzeros
---- Run 1 ----
GF6PTA
TALA
---- Run 2 ----
GF6PTA
TALA
---- Run 3 ----
GF6PTA
FBP
TKT2
---- Run 4 ----
F6PA
GF6PTA
TKT2


We can also instead of using the original objective, specify a given metabolite that we want the model to be able to produce.

In [12]:
with model:
    model.objective = model.add_boundary(model.metabolites.f6p_c, type='demand')
    solution = gapfill(model, universal)
    for reaction in solution[0]:
        print(reaction.id)

Read LP format model from file /var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/tmpl9agwmnw.lp
Reading time = 0.02 seconds
: 2436 rows, 6686 columns, 24998 nonzeros
Read LP format model from file /var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/tmpw6l79cku.lp
Reading time = 0.00 seconds
: 28 rows, 30 columns, 118 nonzeros
PGI


Finally, note that using mixed-integer linear programming is computationally quite expensive and for larger models you may want to consider alternative [gap filling methods](http://opencobra.github.io/cobrapy/tags/gapfilling/) and [reconstruction methods](http://opencobra.github.io/cobrapy/tags/reconstruction/).